In [33]:
import pandas as pd
import difflib
import demoji
import os
import re
from sklearn.metrics import classification_report

In [28]:
# Replace the current path value with the correct path to the GPT-4 dataset directory.
PATH = "/home/lewi/Documents/project/Gpt-4"
data = pd.read_csv(os.path.join(PATH,  'all_amh_LLaMA_eval_responses - all_amh_LLaMA_eval_responses.csv'))
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8089 entries, 0 to 8088
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Instruction  8089 non-null   object
 1   Input Text   8089 non-null   object
 2   Datasource   8089 non-null   object
 3   response     7319 non-null   object
 4   gold_label   8089 non-null   object
dtypes: object(5)
memory usage: 316.1+ KB


In [39]:
# to clean the response 

simmilar_characters = {"ሐ":"ሀ","ሑ":"ሁ","ሒ":"ሂ","ሓ":"ሃ","ሔ":"ሄ","ሕ":"ህ","ሖ":"ሆ",\
                       "ኀ":"ሀ","ኁ":"ሁ","ኂ":"ሂ","ኃ":"ሃ","ኄ":"ሄ","ኅ":"ህ","ኆ":"ሆ",\
                       "ሠ":"ሰ","ሡ":"ሱ","ሢ":"ሲ","ሣ":"ሳ","ሤ":"ሴ","ሥ":"ስ","ሦ":"ሶ","ሧ":"ሷ",\
                       "ዐ":"አ","ዑ":"ኡ","ዒ":"ኢ","ዓ":"ኣ","ዔ":"ኤ","ዕ":"እ","ዖ":"ኦ",\
                       "ጸ":"ፀ","ጹ":"ፁ","ጺ":"ፂ","ጻ":"ፃ","ጼ":"ፄ","ጽ":"ፅ","ጾ":"ፆ"}
 
def clean_am(tweet):
    
    #remove all the english alphabet and numbers
    #tweet = re.sub('[a-zA-Z0-9]', '', tweet)
    tweet = re.sub('[+/..]', ' ', tweet)
    #remove muliple spacing
    tweet = re.sub('\s\s+', '', tweet)
    #remove all the symbols
    tweet = re.sub('[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}፡»~፣።፤-•{…}«፡፡→←]', '', tweet)

    #replace same amharic alphabet
    for letter, althernative in simmilar_characters.items():
           # print(letter, althernative)
            tweet = tweet.replace(letter, althernative)

    return tweet
    
def simmilar_char(text):
    
    for letter, althernative in simmilar_characters.items():
        text = text.replace(letter, althernative)

    return text 

def clean_am_response(df, col_name):
    
    dataframe = df.copy()
    col_name = str(col_name)
    dataframe[col_name] = dataframe[col_name].fillna('NA').astype(str)
    dataframe[col_name] = dataframe[col_name].apply(lambda x: demoji.replace(x,' '))
    dataframe[col_name] = dataframe[col_name].apply(lambda tweet:' '.join([word for word in tweet.split() if len(word)>1]))
    dataframe[col_name] = dataframe[col_name].apply(lambda tweet: tweet.strip())
    dataframe[col_name] = dataframe[col_name].apply(lambda tweet: clean_am(tweet)) 
    dataframe[col_name] = dataframe[col_name].apply(lambda tweet: simmilar_char(tweet)) 
    
    return dataframe[col_name]



def clean_response(df, col_name):
    count = []

    words = ["አዎንታዊ", "አሉታዊ", "ገለልተኛ"]
    for text in df[col_name]:
        for word in words:
            if word in text:
                count.append(word)
                break
        else:
            
            count.append("nothing_predicted")
    return count



In [18]:
def string_similarity(str1, str2):
    return difflib.SequenceMatcher(None, str1, str2).ratio()

def isin_metrics(str1,str2):
    return str1 in str2

In [9]:
for index, row in data.iterrows():
    similarity = string_similarity(row['gold_label'], row['response'])


NameError: name 'string_similarity' is not defined

In [20]:
similarity

0.25806451612903225

In [ ]:
unique_classes = data.gold_label.unique().tolist()
unique_classes += ["nothing_predicted"]
unique_classes

In [77]:
classes = {class_name:index for index,class_name in enumerate(unique_classes)}
y_true = []
y_pred = []




In [23]:
for index, row in data.iterrows():
    y_true.append(classes[row['ouptput']])
    output = [isin_metrics(str1,row['response']) for str1 in classes.keys()]
    
    if True in output:
        y_pred.append(output.index(True))
    else:
        # TODO : how should we handle in the case where we dont find any of the classes
        y_pred.append(classes['nothing_predicted'])


In [49]:
clean_dataset = data[:1999].copy()
answer = pd.DataFrame()
col_name = ['response','gold_label']
for i in col_name:
    
    clean_dataset[i] = clean_am_response(data, i)
    
# clean_dataset = clean_am_response(data, col_name)
for i in col_name:
    
    answer[i] = clean_response(clean_dataset, i)



In [51]:
evalu = (classification_report(clean_dataset['gold_label'], answer['response'], zero_division='warn'))
print(evalu)



                   precision    recall  f1-score   support

nothing_predicted       0.00      0.00      0.00         0
             አሉታዊ       0.79      0.21      0.33      1337
            አዎንታዊ       0.26      0.49      0.34       438
            ገለልተኛ       0.12      0.08      0.10       224

         accuracy                           0.25      1999
        macro avg       0.29      0.19      0.19      1999
     weighted avg       0.60      0.25      0.30      1999



/home/lewi/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lewi/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lewi/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
